<a href="https://colab.research.google.com/github/mnassar/segfault/blob/main/SegFault_TABLE_VII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 9.0MB/s 
     |████████████████████████████████| 174kB 42.2MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import tensorflow.compat.v2 as tf 
tf.enable_v2_behavior()
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from skimage.segmentation import  felzenszwalb, slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import numpy as np
import foolbox as fb
import matplotlib.pyplot as plt

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka' -O CIFAR10model.h5

--2021-06-10 16:46:14--  https://docs.google.com/uc?export=download&id=1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka
Resolving docs.google.com (docs.google.com)... 74.125.142.101, 74.125.142.102, 74.125.142.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cj6atmjq673rl7pmmsdo9ogr84slshur/1623343575000/13345485370729250143/*/1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka?e=download [following]
--2021-06-10 16:46:16--  https://doc-0s-68-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cj6atmjq673rl7pmmsdo9ogr84slshur/1623343575000/13345485370729250143/*/1H4KEE0Vp8DFZOe_QfcxqOxEVnpun-uka?e=download
Resolving doc-0s-68-docs.googleusercontent.com (doc-0s-68-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-0s-68-docs.googleusercontent.com (doc-0s-68-d

In [ ]:
# load the cifar classifier
from tensorflow.keras.models import load_model

pretrained_model = load_model('CIFAR10model.h5')
pretrained_model.trainable = False
pretrained_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)       

In [ ]:
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print (ds_info)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteC5LE2Q/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteC5LE2Q/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cifar10',
    version=3.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'id': Text(shape=(), dtype=tf.string),
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    

In [ ]:
# normalize images 

# these are the numbers used during training the model 
mean = 120.70748
std = 64.150024
bound_min = (0-mean)/std
bound_max = (255-mean)/std
BATCH_SIZE=128

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  # return tf.cast(image, tf.float32) / 255., tf.one_hot(label, 10)
  return (tf.cast(image, tf.float32) - mean) / std, tf.one_hot(label, 10)


ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Dataset Preparation

In [ ]:
NB_BATCHES = 1 # means that we will have NB_BATCHES radnom normal batches
BATCH_SIZE=128
# and NB_BATCHES random adversarial batches each coming from a different normal batch and a different epsilon 

In [ ]:
from keras import backend as K
# pretrained_model.summary()
NB_LAYERS=20
NB_NODES_PER_LAYER=200

In [ ]:
inp = pretrained_model.input  
layers_= pretrained_model.layers[-NB_LAYERS:]
outputs = [lay.output for lay in layers_]     
intermediate_model = K.function([inp], outputs) 


# select NB_NODES_PER_LAYER random nodes from each selected layer
print ("these nodes will be used to compute the IQR-"+str(NB_LAYERS*NB_NODES_PER_LAYER))

node_indices=[]
for lay in layers_[:-1]:
  # we omit the first dim (batch dim) of each layer 
  node_indices.append([[np.random.randint(0,d) for d in lay.output.shape[1:]] for s in range(NB_NODES_PER_LAYER)])
  # print("%s:" % lay.name)
  

# add the last layer 
node_indices.append([[x] for x in range(10)])
# print("%s:" % layers_[-1].name)
print(intermediate_model)

these nodes will be used to compute the IQR-4000
<function function.<locals>.func at 0x7faca412d320>


# IQR calculations with segmentation

In [ ]:
from skimage.segmentation import  felzenszwalb, slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float



In [ ]:
# SLIC experiment 
# number of segments 
# n = 32 
# n = 64 
# n = 128

In [ ]:
# quickshift experiment
# s=1; m=10;
# s=1; m=20;
# s=2; m=10
# s=2; m=20

In [ ]:
# felzenszwalb experiment s=1,10,100,1000
s = 100

In [ ]:


images, labels = iter ( ds_train ) . next () 

preds = pretrained_model.predict(images)
preds_value = tf.reduce_max(preds, axis=1)
preds_idx = tf.argmax(preds, axis=1)
preds_layers = intermediate_model(images)
preds_4k = [] 
for u in range(NB_LAYERS): # loop through the last 10 layers 
  for v in node_indices[u]: # loop through the 10 random nodes for that layer 
    t = tuple(v)
    # print((0,*t))
    preds_4k.append ( preds_layers[u][(...,*t)] )
preds_4k = np.array(preds_4k).T
# print (preds_value)
# print (preds_idx)
# print ( preds_value == tf.gather_nd(preds, list(zip(range(BATCH_SIZE), preds_idx))) )
    
    



In [ ]:
%%time 
iqr = [] 

# choose your seg! 
# segImages = [ slic(img, n_segments=n) for img in images ]
# segImages = [ quickshift(img, sigma=s, max_dist=m) for img in images]
segImages = [ felzenszwalb(img, scale=s) for img in images]

max_n_segs = max (segImg.max() for segImg in segImages)
print (max_n_segs)


for i in range(max_n_segs):     # i is seg number here 
  mask = np.ones((BATCH_SIZE,32,32,3))
  for j in range(BATCH_SIZE): # j is image number here 
    # print (segImages[j]==i)
    # input()
    mask[j,(segImages[j]==i),:] = 0
    
    # print ((segImages[j]==i)[2][3])
    # print (mask[j,2,3,:])
    
  images_0 = images * mask   
  preds_0 = pretrained_model.predict(np.array(images_0))
  preds_value_0 = tf.gather_nd(preds_0, list(zip(range(BATCH_SIZE), preds_idx))) 
  iqr.append(abs(preds_value - preds_value_0)) 

# replace iqr vals of 0 with nan 
iqr = np.array(iqr)
iqr[iqr == 0.0] = np.nan


20
CPU times: user 2.01 s, sys: 69.1 ms, total: 2.08 s
Wall time: 1.96 s


In [ ]:
from sys import getsizeof
getsizeof(iqr)

5232

In [ ]:

%%time 

iqr10 = [] 


# choose your seg! 
# segImages = [ slic(img, n_segments=n) for img in images ]
# segImages = [ quickshift(img, sigma=s, max_dist=m) for img in images]
segImages = [ felzenszwalb(img, scale=s) for img in images]

max_n_segs = max (segImg.max() for segImg in segImages)
print (max_n_segs)

for i in range(max_n_segs):     # i is seg number here 
  mask = np.ones((BATCH_SIZE,32,32,3))
  for j in range(BATCH_SIZE): # j is image number here 
    mask[j,(segImages[j]==i),:] = 0
  images_0 = images * mask 
  
  preds_0 = pretrained_model.predict(np.array(images_0))
  iqr10.append(abs(preds - preds_0))


# for some images the number of segments is less than max_n_segs
# this results in a mask of all ones, the same image will be predicted 
# In this case the difference of predicted value(s) will be 0 
# to ignore these 0s we transform them to NAN and we use NANpercentile function 
# insted of percentile function 

# replace iqr vals of 0 with nan 
iqr10 = np.array(iqr10)
iqr10 [ iqr10 == 0.0] = np.nan

  
    
    
    



10
CPU times: user 767 ms, sys: 36.3 ms, total: 803 ms
Wall time: 743 ms


In [ ]:
from sys import getsizeof
getsizeof(iqr10)

51328

In [ ]:

%%time 


iqr4k = []

# choose your seg! 
# segImages = [ slic(img, n_segments=n) for img in images ]
# segImages = [ quickshift(img, sigma=s, max_dist=m) for img in images]
segImages = [ felzenszwalb(img, scale=s) for img in images]

max_n_segs = max (segImg.max() for segImg in segImages)
print (max_n_segs)

for i in range(max_n_segs):     # i is seg number here 
  mask = np.ones((BATCH_SIZE,32,32,3))
  for j in range(BATCH_SIZE): # j is image number here 
    mask[j,(segImages[j]==i),:] = 0

  images_0 = images * mask 
  
  # preds_0 = pretrained_model.predict(np.array(images_0))
  preds_layers_0 = intermediate_model(images_0)
  preds_4k_0 = [] 
  for u in range(NB_LAYERS): # loop through the last 10 layers ]
      for v in node_indices[u]: # loop through the 10 random nodes for that layer 
        t = tuple(v) 
        preds_4k_0.append ( preds_layers_0[u][(...,*t)] )
  preds_4k_0 = np.array(preds_4k_0).T
  iqr4k.append(abs(preds_4k - preds_4k_0))


# for some images the number of segments is less than max_n_segs
# this results in a mask of all ones, the same image will be predicted 
# In this case the difference of predicted value(s) will be 0 
# to ignore these 0s we transform them to NAN and we use NANpercentile function 
# insted of percentile function 

# replace iqr vals of 0 with nan 
iqr4k = np.array(iqr4k) 
iqr4k[ iqr4k== 0.0] = np.nan

    



10
CPU times: user 1.03 s, sys: 52.7 ms, total: 1.08 s
Wall time: 1.04 s


In [ ]:
from sys import getsizeof
getsizeof(iqr4k)

19507328